# Analysis of Regional Effects on Outcomes
## Data Prep
### For Analysis in Python

In [1]:
# import state-level indicator data
import pandas as pd
state_indicators = pd.read_excel('StateIndicatorsDatabase_2021.xlsx', sheet_name='Data')
# get names of columns for adequacy measures
adequacy_cols = [col for col in state_indicators.columns if col.startswith('necm')]
# select only columns for year, geography, and adequacy
state_indicators = state_indicators[['year', 'state_name', 'region4'] + adequacy_cols]
state_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   1326 non-null   int64  
 1   state_name             1326 non-null   object 
 2   region4                1326 non-null   object 
 3   necm_predcost_state    49 non-null     float64
 4   necm_ppcstot_state     49 non-null     float64
 5   necm_enroll_state      49 non-null     float64
 6   necm_outcomegap_state  49 non-null     float64
 7   necm_fundinggap_state  49 non-null     float64
 8   necm_predcost_q1       48 non-null     float64
 9   necm_ppcstot_q1        48 non-null     float64
 10  necm_enroll_q1         48 non-null     float64
 11  necm_outcomegap_q1     48 non-null     float64
 12  necm_fundinggap_q1     48 non-null     float64
 13  necm_predcost_q2       48 non-null     float64
 14  necm_ppcstot_q2        48 non-null     float64
 15  necm

In [2]:
# filter for 2018
state_indicators_2018 = state_indicators[state_indicators['year'] == 2018]
# drop year (should all be 2018)
state_indicators_2018 = state_indicators_2018.drop('year', axis=1)
# drop states* with missing data: Vermont, Hawaii, D.C.
state_indicators_2018.dropna(how='any', inplace=True)
# view data
state_indicators_2018.head()

,state_name,region4,necm_predcost_state,necm_ppcstot_state,necm_enroll_state,necm_outcomegap_state,necm_fundinggap_state,necm_predcost_q1,necm_ppcstot_q1,necm_enroll_q1,...,necm_predcost_q4,necm_ppcstot_q4,necm_enroll_q4,necm_outcomegap_q4,necm_fundinggap_q4,necm_predcost_q5,necm_ppcstot_q5,necm_enroll_q5,necm_outcomegap_q5,necm_fundinggap_q5
0,Alabama,South,12269.9,9680.97,729626.0,-0.203191,-2588.96,9410.0,9542.0,252990.0,...,14239.0,9530.0,152275.0,-0.410251,-4709.0,17803.0,10166.0,80383.0,-0.555502,-7637.0
1,Alaska,West,12285.3,17665.70,132813.0,-0.131131,5380.42,9958.0,17326.0,24825.0,...,19314.0,31860.0,6840.0,-0.758418,12546.0,25055.0,28288.0,10606.0,-1.096950,3233.0
2,Arizona,West,14123.5,8172.70,931391.0,-0.043099,-5950.81,9408.0,7644.0,351588.0,...,19764.0,8546.0,158241.0,-0.387913,-11218.0,21349.0,9599.0,93179.0,-0.483842,-11750.0
3,Arkansas,South,12392.6,9886.11,465038.0,-0.099895,-2506.50,10730.0,9401.0,160523.0,...,13507.0,9914.0,46521.0,-0.189624,-3593.0,15298.0,10599.0,76969.0,-0.388631,-4699.0
4,California,West,16897.8,12077.10,5941124.0,-0.199705,-4820.73,10616.0,11672.0,1173054.0,...,20309.0,12656.0,1851923.0,-0.438514,-7653.0,23276.0,12631.0,625823.0,-0.622818,-10645.0


### Regional Aggregation (for Power BI)

In [3]:
# middle of names of variables averaged per pupil 
cols_to_scale = ['ppcstot', 'predcost', 'outcomegap', 'fundinggap']
# endings of variable names indicating different aggregate levels
endings = ['state', 'q1', 'q2', 'q3', 'q4', 'q5']

weighted_indicators = state_indicators_2018.copy()

# create columns to weigh per-pupil values by the enrollment and drop original columns
for col in cols_to_scale:
    for ending in endings:
        # add column weighted by enrollment
        weighted_indicators[f'weighted_{col}_{ending}'] = state_indicators_2018[f'necm_{col}_{ending}'] * state_indicators_2018[f'necm_enroll_{ending}']
        # drop unweighted column
        weighted_indicators.drop(f'necm_{col}_{ending}', axis=1, inplace=True)

weighted_indicators.head()

,state_name,region4,necm_enroll_state,necm_enroll_q1,necm_enroll_q2,necm_enroll_q3,necm_enroll_q4,necm_enroll_q5,weighted_ppcstot_state,weighted_ppcstot_q1,...,weighted_outcomegap_q2,weighted_outcomegap_q3,weighted_outcomegap_q4,weighted_outcomegap_q5,weighted_fundinggap_state,weighted_fundinggap_q1,weighted_fundinggap_q2,weighted_fundinggap_q3,weighted_fundinggap_q4,weighted_fundinggap_q5
0,Alabama,South,729626.0,252990.0,159949.0,84029.0,152275.0,80383.0,7.063487e+09,2.414031e+09,...,-22306.007693,-22986.132950,-62470.971025,-44652.917266,-1.888973e+09,3.339468e+07,-2.890278e+08,-3.022523e+08,-7.170630e+08,-6.138850e+08
1,Alaska,West,132813.0,24825.0,82598.0,7944.0,6840.0,10606.0,2.346235e+09,4.301180e+08,...,-557.949490,-317.450184,-5187.579120,-11634.251700,7.145897e+08,1.829106e+08,3.996917e+08,1.192394e+07,8.581464e+07,3.428920e+07
2,Arizona,West,931391.0,351588.0,184564.0,143819.0,158241.0,93179.0,7.611979e+09,2.687539e+09,...,-9298.334320,-36317.461518,-61383.741033,-45083.913718,-5.542531e+09,-6.202012e+08,-1.052015e+09,-1.000261e+09,-1.775148e+09,-1.094853e+09
3,Arkansas,South,465038.0,160523.0,79991.0,101034.0,46521.0,76969.0,4.597417e+09,1.509077e+09,...,-3219.877723,-18712.507140,-8821.498104,-29912.539439,-1.165618e+09,-2.133351e+08,-1.129473e+08,-3.105785e+08,-1.671500e+08,-3.616773e+08
4,California,West,5941124.0,1173054.0,1044816.0,1245508.0,1851923.0,625823.0,7.175155e+10,1.369189e+10,...,-98561.672544,-254104.805636,-812094.162422,-389773.829214,-2.864055e+10,1.238745e+09,-2.826227e+09,-6.220067e+09,-1.417277e+10,-6.661886e+09


In [4]:
# sum values by region
regions_2018 = pd.DataFrame(weighted_indicators.groupby('region4').sum())

# scale values by enrollment to get averages reflecting student population
for col in cols_to_scale:
    for ending in endings:
        # add column weighted by enrollment
        regions_2018[f'{col}_{ending}'] = regions_2018[f'weighted_{col}_{ending}'] / regions_2018[f'necm_enroll_{ending}']
        # drop unweighted column
        regions_2018.drop(f'weighted_{col}_{ending}', axis=1, inplace=True)

regions_2018.head()

,necm_enroll_state,necm_enroll_q1,necm_enroll_q2,necm_enroll_q3,necm_enroll_q4,necm_enroll_q5,ppcstot_state,ppcstot_q1,ppcstot_q2,ppcstot_q3,...,outcomegap_q2,outcomegap_q3,outcomegap_q4,outcomegap_q5,fundinggap_state,fundinggap_q1,fundinggap_q2,fundinggap_q3,fundinggap_q4,fundinggap_q5
region4,,,,,,,,,,,,,,,,,,,,,
Midwest,9942544.0,2907961.0,1698164.0,1421992.0,1497249.0,2417178.0,12176.873953,11947.655726,11825.787452,11831.520502,...,0.193059,0.076045,-0.034681,-0.298714,1419.575065,5658.956820,3022.992256,1361.692857,-419.946032,-3633.452288
Northeast,7312623.0,1447614.0,1098306.0,979829.0,976780.0,2810094.0,19502.329481,19253.408173,18800.291272,17963.541306,...,0.394399,0.256086,0.068998,-0.181527,6575.296688,12600.642505,10392.093156,8584.450518,6352.007781,1356.311446
South,18989652.0,6353575.0,4587187.0,3438530.0,2317217.0,2293143.0,10212.872241,9952.523303,10177.778457,10460.282905,...,0.014343,-0.085205,-0.200747,-0.270816,-2156.325545,717.017866,-1753.083630,-2687.843366,-4763.758493,-7492.200980
West,11460713.0,3018402.0,2238054.0,2364884.0,2498166.0,1341207.0,11219.708144,10565.314531,10845.178063,11298.836845,...,-0.026240,-0.149437,-0.370939,-0.437801,-3387.424138,1077.951756,-1673.563442,-4008.434572,-6915.454904,-8630.979017


In [5]:
# export data to csv
regions_2018.to_csv('regions_2018.csv')

## ANOVA and Tukey Tests
### Create Function
Create function to perform ANOVA and Tukey Tests to test if there is a statistical significance between the regional means of a given indicator.

In [6]:
# Import packages for ANOVA and Tukey Tests
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def anova_and_tukey_by_region(indicator):
    '''perform ANOVA and Tukey Test between regions for the given indicator'''
    # Get outcome data by region
    northeast = state_indicators_2018[state_indicators_2018['region4'] == 'Northeast'][indicator]
    south = state_indicators_2018[state_indicators_2018['region4'] == 'South'][indicator]
    west = state_indicators_2018[state_indicators_2018['region4'] == 'West'][indicator]
    midwest = state_indicators_2018[state_indicators_2018['region4'] == 'Midwest'][indicator]

    # perform one-way ANOVA on regional outcomes - pvalue is significant
    print('ANOVA: ', f_oneway(northeast, south, west, midwest))

    tukey = pairwise_tukeyhsd(endog=state_indicators_2018[indicator], groups=state_indicators_2018['region4'], alpha=0.05)
    print(tukey)

### Apply Function
#### State-level outcome gap

In [7]:
# State-Level Outcome Gap
anova_and_tukey_by_region('necm_outcomegap_state')

ANOVA:  F_onewayResult(statistic=6.714236820243193, pvalue=0.0007897911952486875)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  Midwest Northeast   0.0736 0.5534  -0.076  0.2232  False
  Midwest     South  -0.1342 0.0313 -0.2594 -0.0091   True
  Midwest      West  -0.1094 0.1435 -0.2432  0.0243  False
Northeast     South  -0.2078 0.0017 -0.3497 -0.0659   True
Northeast      West   -0.183 0.0109 -0.3326 -0.0334   True
    South      West   0.0248    0.9 -0.1004  0.1499  False
----------------------------------------------------------


#### Outcome gap for lowest poverty districts (Q1)

In [8]:
anova_and_tukey_by_region('necm_outcomegap_q1')

ANOVA:  F_onewayResult(statistic=19.505426678411666, pvalue=3.4249489326231416e-08)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  Midwest Northeast    0.213 0.0195  0.0265  0.3995   True
  Midwest     South  -0.2407  0.001 -0.3967 -0.0846   True
  Midwest      West  -0.2118 0.0078 -0.3786  -0.045   True
Northeast     South  -0.4537  0.001 -0.6306 -0.2768   True
Northeast      West  -0.4248  0.001 -0.6113 -0.2383   True
    South      West   0.0289    0.9 -0.1272  0.1849  False
----------------------------------------------------------


#### Outcome gap for low poverty districts (Q2)

In [9]:
anova_and_tukey_by_region('necm_outcomegap_q2')

ANOVA:  F_onewayResult(statistic=18.975829582001676, pvalue=4.8027696321227915e-08)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  Midwest Northeast   0.2221  0.007  0.0492   0.395   True
  Midwest     South  -0.2072 0.0022 -0.3519 -0.0625   True
  Midwest      West   -0.163 0.0353 -0.3177 -0.0084   True
Northeast     South  -0.4293  0.001 -0.5933 -0.2652   True
Northeast      West  -0.3851  0.001  -0.558 -0.2122   True
    South      West   0.0442 0.8291 -0.1005  0.1888  False
----------------------------------------------------------


#### Outcome gap for medium poverty districts (Q3)

In [10]:
anova_and_tukey_by_region('necm_outcomegap_q3')

ANOVA:  F_onewayResult(statistic=16.222965829989953, pvalue=3.039705198124666e-07)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  Midwest Northeast   0.1745 0.0526 -0.0014  0.3503  False
  Midwest     South  -0.2243 0.0011 -0.3715 -0.0772   True
  Midwest      West  -0.1656 0.0357 -0.3229 -0.0083   True
Northeast     South  -0.3988  0.001 -0.5657 -0.2319   True
Northeast      West    -0.34  0.001 -0.5159 -0.1641   True
    South      West   0.0588 0.6901 -0.0884  0.2059  False
----------------------------------------------------------


#### Outcome gap for high poverty districts (Q4)

In [11]:
anova_and_tukey_by_region('necm_outcomegap_q4')

ANOVA:  F_onewayResult(statistic=5.697568185536517, pvalue=0.0021934596296669263)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  Midwest Northeast   0.1708 0.2222 -0.0629  0.4045  False
  Midwest     South  -0.1403 0.2365 -0.3358  0.0552  False
  Midwest      West  -0.1307 0.3522 -0.3397  0.0783  False
Northeast     South  -0.3111 0.0028 -0.5328 -0.0894   True
Northeast      West  -0.3015 0.0067 -0.5352 -0.0678   True
    South      West   0.0096    0.9 -0.1859  0.2051  False
----------------------------------------------------------


#### Outcome gap for highest poverty districts (Q5)

In [12]:
anova_and_tukey_by_region('necm_outcomegap_q5')

ANOVA:  F_onewayResult(statistic=1.241778810556407, pvalue=0.30602807765564294)
   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
  group1    group2  meandiff p-adj   lower  upper  reject
---------------------------------------------------------
  Midwest Northeast   0.1043 0.6346 -0.1344 0.3429  False
  Midwest     South  -0.0083    0.9  -0.208 0.1914  False
  Midwest      West  -0.0677 0.8115 -0.2812 0.1457  False
Northeast     South  -0.1126 0.5457  -0.339 0.1139  False
Northeast      West   -0.172  0.233 -0.4106 0.0667  False
    South      West  -0.0594 0.8404 -0.2591 0.1402  False
---------------------------------------------------------
